# Volume

In [0]:
%sql

SELECT *
FROM read_files(
  '/Volumes/workspace/raw/rawvolume/rawdata/customers/dim_passengers.csv',
  format => 'csv',
  header => true,
  inferSchema => true
);

In [0]:
%sql

SELECT *
FROM read_files(
  '/Volumes/workspace/raw/rawvolume/rawdata/airports/dim_airports.csv',
  format => 'csv',
  header => true,
  inferSchema => true
);

In [0]:
%sql
SELECT *
FROM read_files(
  '/Volumes/workspace/raw/rawvolume/rawdata/airports/dim_airports.csv',
  format => 'csv',
  header => true,
  inferSchema => true
);

# Bronze Layer

In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/bronze/bronzevolume/bookings/data`;--Run-1: 1000 rows. Run-2: 1300 Rows (Incremental Load)

In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/bronze/bronzevolume/customers/data`;--Run-1: 200 rows. Run-2: 220 Rows (Incremental Load); --235(SCD Load)

In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/bronze/bronzevolume/airports/data`;--Run-1: 50 rows. Run-2: 55 Rows (Incremental Load); --61 rows(Scd Load)

In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/bronze/bronzevolume/flights/data`;--Run-1: 100 rows. Run-2: 110 Rows (Incremental Load);--118 Rows(Scd Load)

# Silver Layer

In [0]:
%sql
SELECT * FROM workspace.silver.stage_bookings--1300 rows

In [0]:
%sql
SELECT * FROM workspace.silver.silver_bookings;--1300 rows

In [0]:
%sql
SELECT * FROM workspace.silver.silver_flights--110 rows; --113 ros(scd Load)

In [0]:
%sql
SELECT * FROM workspace.silver.silver_customers--220 rows; --225 rows(Scd Load)

In [0]:
%sql
SELECT * FROM workspace.silver.silver_airports--55 rows; 58 rows(scd load)

In [0]:
%sql
SELECT * FROM workspace.silver.silver_business;--1300 Rows

In [0]:
%sql
SELECT * FROM workspace.silver.silver_business_mat;--1300 Rows

# Gold Layer

In [0]:
%sql
select * from workspace.gold.dimflights;--110(First Run); --113 rows(Scd Load)

In [0]:
%sql
select * from workspace.gold.dimairports;--55(First Run); --58 rows(scd Load)

In [0]:
%sql
select * from workspace.gold.dimcustomers;--220(First Run); --225 roews(scd Load)

In [0]:
%sql
--Scd Data Checks

SELECT *
FROM read_files(
  '/Volumes/workspace/raw/rawvolume/rawdata/customers/dim_passengers_scd.csv',
  format => 'csv',
  header => true,
  inferSchema => true
)
WHERE passenger_id NOT IN (
  SELECT passenger_id
  FROM read_files(
    '/Volumes/workspace/raw/rawvolume/rawdata/customers/dim_passengers.csv',
    format => 'csv',
    header => true,
    inferSchema => true
  )
)--5 rows(These are Inserted newly in scd load in silver and gold)

### Above Query Written in Python

In [0]:
# Read the SCD CSV file
df_scd = (spark.read.format("csv")
          .option("header", "true")
          .option("inferSchema", "true")
          .load("/Volumes/workspace/raw/rawvolume/rawdata/customers/dim_passengers_scd.csv"))

# Read the incremental CSV file
df_increment = (spark.read.format("csv")
                .option("header", "true")
                .option("inferSchema", "true")
                .load("/Volumes/workspace/raw/rawvolume/rawdata/customers/dim_passengers.csv"))

# Filter the SCD DataFrame to exclude passenger_ids present in the incremental DataFrame
df_result = df_scd.join(df_increment, on="passenger_id", how="left_anti")

# Display the result
display(df_result)

In [0]:
%sql
--Scd Data Checks

SELECT *
FROM read_files(
  '/Volumes/workspace/raw/rawvolume/rawdata/airports/dim_airports_scd.csv',
  format => 'csv',
  header => true,
  inferSchema => true
)
WHERE airport_id NOT IN (
  SELECT airport_id
  FROM read_files(
    '/Volumes/workspace/raw/rawvolume/rawdata/airports/dim_airports.csv',
    format => 'csv',
    header => true,
    inferSchema => true
  )
)--3 rows(These are Inserted newly in scd load in silver and gold)

In [0]:
%sql
--Scd Data Checks

SELECT *
FROM read_files(
  '/Volumes/workspace/raw/rawvolume/rawdata/flights/dim_flights_scd.csv',
  format => 'csv',
  header => true,
  inferSchema => true
)
WHERE flight_id NOT IN (
  SELECT flight_id
  FROM read_files(
    '/Volumes/workspace/raw/rawvolume/rawdata/flights/dim_flights.csv',
    format => 'csv',
    header => true,
    inferSchema => true
  )
)--3 rows(These are Inserted newly in scd load in silver and gold)